In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
from numba import cuda
import math

#### Obtain and reshape training and test data

In [ ]:
download_data = True

In [ ]:
# Define transform to convert data to tensors
transform = transforms.Compose([transforms.ToTensor()])

# Download MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=download_data, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=download_data, transform=transform)

# Convert to NumPy arrays
x_train = train_dataset.data.numpy()
y_train = train_dataset.targets.numpy()
x_test = test_dataset.data.numpy()
y_test = test_dataset.targets.numpy()

# Reshape the data
x_train_flatten = x_train.reshape(x_train.shape[0],-1).T / 255.  # The "-1" makes reshape flatten the remaining dimensions
x_test_flatten = x_test.reshape(x_test.shape[0],-1).T /255. # regularize data by 1/255
y_train_flatten = y_train.reshape(y_train.T.shape[0],1).T
y_test_flatten = y_test.reshape(y_test.T.shape[0],1).T 

#### CPU Functions on host

In [ ]:
def init_params() :
    W1 = np.random.uniform (-0.15,0.15, (n_hidden_nodes, 784))
    b1 = np.zeros((n_hidden_nodes, 1))
    W2 = np.random.uniform (-0.15,0.15, (10, n_hidden_nodes))
    b2 = np.zeros((10, 1))
    return W1, b1, W2, b2

# these perhaps need rewriting too:
def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions,Y):
    #print(predictions,Y)
    return np.sum(predictions==Y)/Y.size

def one_hot_(Y) :
    Y = Y.flatten()  # Convert shape (1, m) to (m,)
    one_hot_Y = np.zeros((10, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1  # Fix indexing order
    return one_hot_Y    

#### Helper device functions

In [ ]:
@cuda.jit(device=True)
def relu_gpu(x):
    return max(0, x)

@cuda.jit(device=True)
def deriv_relu_gpu(x):
    return 1.0 if x > 0 else 0.0

#### Cuda kernels

In [ ]:
@cuda.jit
def relu_elementwise_gpu(Z, A):
    i, j = cuda.grid(2)
    if i < Z.shape[0] and j < Z.shape[1]:
        A[i, j] = relu_gpu(Z[i, j])

@cuda.jit
def deriv_relu_elementwise_gpu(Z, dA):
    i, j = cuda.grid(2)
    if i < Z.shape[0] and j < Z.shape[1]:
        dA[i, j] = deriv_relu_gpu(Z[i, j])

@cuda.jit
def matmul_elementwise_gpu(A, B, C):
    row, col = cuda.grid(2)
    if row < A.shape[0] and col < B.shape[1]:
        tmp = 0.0
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

@cuda.jit
def softmax_elementwise_gpu(Z, A):
    col = cuda.grid(1)
    if col < Z.shape[1]:
        max_val = -float('inf')
        for i in range(Z.shape[0]):
            if Z[i, col] > max_val:
                max_val = Z[i, col]
        sum_exp = 0.0
        temp_exp = cuda.local.array(10, dtype=np.float32)
        for i in range(Z.shape[0]):
            temp_exp[i] = math.exp(Z[i, col] - max_val)
            sum_exp += temp_exp[i]
        for i in range(A.shape[0]):
            A[i, col] = temp_exp[i] / sum_exp


#### NN functions - to be rewritten

In [ ]:
def forward_prop(W1,b1,W2,b2,X) :
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softMAX(Z2)
    return Z1, A1, Z2, A2

def back_prop(Z1, A1, Z2, A2, W2, X, Y) :
    m = Y.size
    one_hot_Y = one_hot_(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha) :
    W1 -= alpha * dW1
    b1 -= alpha * np.reshape(db1,(n_hidden_nodes,1))
    W2 -= alpha * dW2
    b2 -= alpha * np.reshape(db2,(10,1))
    return W1, b1, W2, b2

def gradient_descent(X, Y, X_, Y_, iterations, alpha):
    iterations+=1
    for i in range(iterations) :
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # test results:
        _,  _,  _,  A2_= forward_prop(W1, b1, W2, b2, X_)
        if i % 50 == 0 :
            print("Itertation:",i,"Train Accuracy:", get_accuracy(get_predictions(A2),Y))
            print("Itertation:",i,"Test Accuracy:", get_accuracy(get_predictions(A2_),Y_))
    return W1, b1, W2, b2

#### Do actual training

In [ ]:
# hyper parameters
n_hidden_nodes = 100
n_iterations = 200
learning_rate = 0.3

In [ ]:
W1, b1, W2, b2 = init_params()

# move data to the GPU
W1_d = cuda.to_device(W1)
b1_d = cuda.to_device(b1)
W2_d = cuda.to_device(W2)
b2_d = cuda.to_device(b2)
X_train_d = cuda.to_device(x_train_flatten)
X_test_d = cuda.to_device(x_test_flatten)
Y_train_d = cuda.to_device(one_hot_(y_train_flatten))
Y_test_d = cuda.to_device(one_hot_(y_test_flatten))


# run training...
W1, b1, W2, b2 = gradient_descent...(X_train_d, Y_train_d, X_test_d, Y_test_d, n_iterations, learning_rate)